<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [2]:
import numpy as np
import pandas as pd
import lxml.html 

*It is good practice to display the library version numbers for future reference:*

In [6]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.19.2
Pandas:  1.1.5


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [26]:
# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = None)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 1)
print(bikes.head())
print()

# wrong:
bikes = pd.read_table('dat/bikeshare.csv', header = 0)
print(bikes.head())

FileNotFoundError: [Errno 2] No such file or directory: 'dat/bikeshare.csv'

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [7]:
#ANSWER:
bikes = pd.read_table('bikeshare.csv')
bikes.head()

,"instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt"
0,"1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,..."
1,"2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0..."
2,"3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0..."
3,"4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,..."
4,"5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,..."


In [8]:
bikes = pd.read_csv('bikeshare.csv')
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [12]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [15]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [0]:
#ANSWER


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [16]:
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2021-01-02T00:42:18.458Z,13.7464,-91.2052,35.00,4.3,mb,NaN,159,0.907,1.15,...,2021-01-02T01:05:55.402Z,"40 km SSW of La Gomera, Guatemala",earthquake,8.1,2.0,0.049,117,reviewed,us,us
1,2021-01-02T00:26:19.618Z,-34.4012,-70.5107,124.54,4.6,mb,NaN,127,0.419,0.63,...,2021-01-02T00:53:47.042Z,"27 km SSE of Machalí, Chile",earthquake,4.7,5.2,0.136,16,reviewed,us,us


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [3]:
url = 'http://www.fdic.gov/bank/individual/failed/banklist.html'
df = pd.read_html(url)
df

[                             Bank Name               City  ST   CERT  \
 0                    Almena State Bank             Almena  KS  15426   
 1           First City Bank of Florida  Fort Walton Beach  FL  16748   
 2                 The First State Bank      Barboursville  WV  14361   
 3                   Ericson State Bank            Ericson  NE  18265   
 4     City National Bank of New Jersey             Newark  NJ  21111   
 ..                                 ...                ...  ..    ...   
 558                 Superior Bank, FSB           Hinsdale  IL  32646   
 559                Malta National Bank              Malta  OH   6629   
 560    First Alliance Bank & Trust Co.         Manchester  NH  34264   
 561  National State Bank of Metropolis         Metropolis  IL   3815   
 562                   Bank of Honolulu           Honolulu  HI  21029   
 
                    Acquiring Institution       Closing Date  
 0                            Equity Bank   October 23, 202

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [4]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient = 'columns')
df.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [11]:
bikes.shape

(17379, 17)

What are the column names in `bikes`?

In [12]:
bikes.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

What are the data types of these columns?

In [15]:
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [21]:
bikes[:0]

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt


https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [22]:
print(bikes['season'])
print(bikes[['season']])

0        1
1        1
2        1
3        1
4        1
        ..
17374    1
17375    1
17376    1
17377    1
17378    1
Name: season, Length: 17379, dtype: int64
       season
0           1
1           1
2           1
3           1
4           1
...       ...
17374       1
17375       1
17376       1
17377       1
17378       1

[17379 rows x 1 columns]


How would we use object notation to show the first 4 rows of `atemp`?

In [28]:
bikes.atemp.iloc[0:4]

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [33]:
#E bikes[[0]]
#E bikes[0]
#E bikes[0,0]
bikes[[0,0]]

KeyError: "None of [Int64Index([0, 0], dtype='int64')] are in the [columns]"

What is the correct way to access the 1st row of the DataFrame by its index?

In [35]:
bikes.loc[0]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed              0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [39]:
bikes.iloc[:, 1]

0        2011-01-01
1        2011-01-01
2        2011-01-01
3        2011-01-01
4        2011-01-01
            ...    
17374    2012-12-31
17375    2012-12-31
17376    2012-12-31
17377    2012-12-31
17378    2012-12-31
Name: dteday, Length: 17379, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

?
ANSWER: To count how many null values are in a given column

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [40]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [46]:
print(bikes.isnull().sum().sum())

0


If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [45]:
bikes.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

What is the Numpy object `nan` used for? (Write a descriptive answer.)

?
ANSWER: Marking a data point as invalid - as there is no value in the given cell

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [50]:
def div_q(a, b):
    if b == 0:
         print('This results in an error')
    
    else:
        result = (a / b)
        print(result)
    

In [58]:
div_q(10, 4)
div_q(10,0)
div_q(30,3)
div_q(9000,0)

2.5
This results in an error
10.0
This results in an error


Apply the Pandas `isna` function to the following data objects:

In [59]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [64]:
#ANSWER
pd.isna([x, y])

array([False,  True])

In [61]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [62]:
#ANSWER
pd.isna(array)

array([[False,  True, False],
       [False, False,  True]])

How is the pandas I/O parameter `na_values` used?

ANSWER: na_values is used to consider specific values as na when parsing a DataFrame.

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [80]:
#ANSWER
print(bikes.season.unique())
print(bikes.season.value_counts())

[1 2 3 4]
3    4496
2    4409
1    4242
4    4232
Name: season, dtype: int64


### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [105]:
#ANSWER
print(bikes.instant.max() - bikes.instant.min())
print(bikes.windspeed.max() - bikes.windspeed.min())


17378
0.8507


In [106]:
print(bikes.dteday.max())
print(bikes.dteday.min())
#ANSWER: Range = 2 years

2012-12-31
2011-01-01


Compute and print the overall minimum and maximum of the numeric data columns:

In [82]:
bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max

# Answer below

TypeError: '>' not supported between instances of 'numpy.ndarray' and 'str'

In [91]:
bikes_min = bikes.min()
print(bikes_min)

print(bikes.max())

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                0
workingday             0
weathersit             1
temp                0.02
atemp                  0
hum                    0
windspeed              0
casual                 0
registered             0
cnt                    1
dtype: object
instant            17379
dteday        2012-12-31
season                 4
yr                     1
mnth                  12
hr                    23
holiday                1
weekday                6
workingday             1
weathersit             4
temp                   1
atemp                  1
hum                    1
windspeed         0.8507
casual               367
registered           886
cnt                  977
dtype: object


### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [83]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [109]:
#ANSWER
bikes.quantile([0.1, 0.25, 0.5, 0.75, 0.9])

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0.10,1738.8,1.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.24,0.2424,0.37,0.0000,1.0,7.0,9.0
0.25,4345.5,2.0,0.0,4.0,6.0,0.0,1.0,0.0,1.0,0.34,0.3333,0.48,0.1045,4.0,34.0,40.0
0.50,8690.0,3.0,1.0,7.0,12.0,0.0,3.0,1.0,1.0,0.50,0.4848,0.63,0.1940,17.0,115.0,142.0
0.75,13034.5,3.0,1.0,10.0,18.0,0.0,5.0,1.0,2.0,0.66,0.6212,0.78,0.2537,48.0,220.0,281.0
0.90,15641.2,4.0,1.0,11.0,21.0,0.0,6.0,1.0,2.0,0.74,0.6970,0.88,0.3582,92.0,354.0,451.2


In [111]:
#Checking the above table of percentiles calculated correctly
bikes.max()

instant            17379
dteday        2012-12-31
season                 4
yr                     1
mnth                  12
hr                    23
holiday                1
weekday                6
workingday             1
weathersit             4
temp                   1
atemp                  1
hum                    1
windspeed         0.8507
casual               367
registered           886
cnt                  977
dtype: object

In [113]:
#Not sure if the question is asking for atemp specifically:
bikes.atemp.quantile([0.1, 0.25, 0.5, 0.75, 0.9])

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [114]:
type(bikes['atemp'])

pandas.core.series.Series

In [115]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
8447,8448,2011-12-23,1,0,12,15,0,5,1,1,0.38,0.3939,0.54,0.3582,12,191,203
10889,10890,2012-04-03,2,1,4,21,0,2,1,1,0.54,0.5152,0.45,0.0000,33,210,243
15180,15181,2012-09-29,4,1,9,17,0,6,0,1,0.60,0.6212,0.38,0.3582,256,456,712
5133,5134,2011-08-06,3,0,8,19,0,6,0,3,0.72,0.7121,0.84,0.2537,57,114,171
14686,14687,2012-09-09,3,1,9,3,0,0,0,1,0.54,0.5152,0.73,0.2239,14,32,46


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [116]:
atemp_level = pd.cut(bikes['atemp'], bins = 4)         

What is `atemp_level`?

In [120]:
#ANSWER: A classification column to divide and assign the rows of the DataFrame into 4 equal groups based on the values of the atemp column

Here is a random sample of `atemp_level`:

In [119]:
atemp_level.sample(5)          

1530        (0.25, 0.5]
10592       (0.25, 0.5]
13937       (0.5, 0.75]
9515     (-0.001, 0.25]
3571        (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [121]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

5094     warm
4242     warm
11814    warm
7504     mild
5420     warm
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [124]:
#ANSWER
bikes['atemp_level'] = atemp_level
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,atemp_level
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,mild
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,mild
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,mild
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,mild
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,mild


In [153]:
seasons_sorted = bikes.sort_values('season')
season2_sorted = bikes.loc[bikes.season == 2]
season2_mild = season2_sorted.loc[season2_sorted.atemp_level == 'mild'] 
season2_mild_total = season2_mild.value_counts()
print(season2_mild_total)

instant  dteday      season  yr  mnth  hr  holiday  weekday  workingday  weathersit  temp  atemp   hum   windspeed  casual  registered  cnt  atemp_level
12699    2012-06-18  2       1   6     7   0        1        1           3           0.52  0.5000  0.94  0.2239     10      135         145  mild           1
2594     2011-04-22  2       0   4     23  0        5        1           3           0.32  0.3333  0.93  0.1045     0       17          17   mild           1
2596     2011-04-23  2       0   4     1   0        6        0           2           0.32  0.3182  1.00  0.1940     5       11          16   mild           1
2597     2011-04-23  2       0   4     2   0        6        0           3           0.32  0.3333  1.00  0.1343     3       14          17   mild           1
2598     2011-04-23  2       0   4     3   0        6        0           3           0.32  0.3333  1.00  0.0896     0       4           4    mild           1
                                                         

In [155]:
season2_mild_total.shape

(1829,)

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [156]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [164]:
# Cleaning Data
print(a.shape, a.dtype)
print(b.shape, b.dtype)
print(c.shape, c.dtype)
print(d.shape, d.dtype)
print(e.shape, e.dtype)

(3,) float64
(1, 3) float64
(2, 3, 4) int32
(4,) int32
(2, 4) int32


## Load Data

Load rock.csv and clean the dataset.

In [209]:
rockdf = pd.read_csv('rock.csv')
rockdf.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Check Column Names

Check column names and clean.

In [210]:
rockdf.columns

Index(['Song Clean', 'ARTIST CLEAN', 'Release Year', 'COMBINED', 'First?',
       'Year?', 'PlayCount', 'F*G'],
      dtype='object')

# Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [228]:
rockdf.isnull().sum()

Song Clean      0
ARTIST CLEAN    0
Release Year    0
COMBINED        0
First?          0
Year?           0
PlayCount       0
F*G             0
dtype: int64

In [226]:
rockdf = rockdf.fillna(0, inplace=False)
print(rockdf)

                                       Song Clean ARTIST CLEAN  Release Year  \
0                                Caught Up in You  .38 Special        1982.0   
1                                    Fantasy Girl  .38 Special           0.0   
2                                 Hold On Loosely  .38 Special        1981.0   
3                          Rockin' Into the Night  .38 Special        1980.0   
4                               Art For Arts Sake         10cc        1975.0   
...                                           ...          ...           ...   
2225                      She Loves My Automobile       ZZ Top           0.0   
2226                            Tube Snake Boogie       ZZ Top        1981.0   
2227                                         Tush       ZZ Top        1975.0   
2228                                   TV Dinners       ZZ Top        1983.0   
2229  WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO       ZZ Top        1973.0   

                                       

# Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [227]:
rockdf.dtypes

Song Clean       object
ARTIST CLEAN     object
Release Year    float64
COMBINED         object
First?            int64
Year?             int64
PlayCount         int64
F*G               int64
dtype: object

In [223]:
rockdf['Release Year'] = pd.to_numeric(rockdf['Release Year'], errors= 'coerce')

In [229]:
rockdf.iloc[1504]

Song Clean                        Bullfrog Blues
ARTIST CLEAN                      Rory Gallagher
Release Year                                   0
COMBINED        Bullfrog Blues by Rory Gallagher
First?                                         1
Year?                                          1
PlayCount                                      1
F*G                                            1
Name: 1504, dtype: object

In [221]:
# Not used - turned SONGFACTS.COM into NaN using errors = 'coerce' above, then turned into 0 with fillna
rockdf.replace('SONGFACTS.COM', 0)

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2225,She Loves My Automobile,ZZ Top,0,She Loves My Automobile by ZZ Top,1,0,1,0
2226,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2227,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2228,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


# Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [231]:
print(rockdf.min())
print(rockdf.max())

Song Clean                     #9 Dream
ARTIST CLEAN                .38 Special
Release Year                          0
COMBINED        #9 Dream by John Lennon
First?                                1
Year?                                 0
PlayCount                             0
F*G                                   0
dtype: object
Song Clean                     Ziggy Stardust
ARTIST CLEAN                             a-ha
Release Year                             2014
COMBINED        Ziggy Stardust by David Bowie
First?                                      1
Year?                                       1
PlayCount                                 142
F*G                                       142
dtype: object


In [232]:
rockdf.head()

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982.0,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,0.0,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981.0,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980.0,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975.0,Art For Arts Sake by 10cc,1,1,1,1


# Write Some Functions

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [265]:
Song_info = input("Enter row number: ")
Song = rockdf['Song Clean']
Artist = rockdf['ARTIST CLEAN']
Release_Year = rockdf['Release Year']
print(Song + " by " + Artist + str(Release_Year))

Enter row number: 56
0       Caught Up in You by .38 Special0       1982.0\...
1       Fantasy Girl by .38 Special0       1982.0\n1  ...
2       Hold On Loosely by .38 Special0       1982.0\n...
3       Rockin' Into the Night by .38 Special0       1...
4       Art For Arts Sake by 10cc0       1982.0\n1    ...
                              ...                        
2225    She Loves My Automobile by ZZ Top0       1982....
2226    Tube Snake Boogie by ZZ Top0       1982.0\n1  ...
2227    Tush by ZZ Top0       1982.0\n1          0.0\n...
2228    TV Dinners by ZZ Top0       1982.0\n1         ...
2229    WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO by...
Length: 2230, dtype: object


In [ ]:
Song_info = input("Enter row number: ")
Song = rockdf
print(Song)

In [249]:
def func:
    
    
    Song = rockdf['Song Clean']
    Artist = rockdf['ARTIST CLEAN']
    Release_Year = rockdf['Release Year']
    
        if Release_Year < 1970:
            print('Release Year < 1970')
        else:
            print("Release Year > 1970")
    
    return (func)

IndentationError: unexpected indent (<ipython-input-249-d2abc10b228d>, line 6)

In [259]:
rockdf['ARTIST CLEAN'].sample(1)

1496    Rolling Stones
Name: ARTIST CLEAN, dtype: object

Song Clean                       Patience
ARTIST CLEAN                Guns N' Roses
Release Year                         1988
COMBINED        Patience by Guns N' Roses
First?                                  1
Year?                                   1
PlayCount                              21
F*G                                    21
Name: 759, dtype: object

## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

## Apply these functions to your dataset

## Describe the new float-only DataFrame.



> 
>
>




>


>


>




---



---



> > > > > > > > > © 2019 Institute of Data


---



---



